# Generation of Configuration

## Setup

In [ ]:
val ivy_path = System.getProperty("user.dir") + "/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(ivy_path)))

In [ ]:
import pillars.core._
import pillars.mapping.thirdParty._
import pillars.archlib.TileLSUBlock
import pillars.hardware.{TopModule}
import pillars.archlib._
import pillars.core.{BlockTrait, OpEnum}
import pillars.core.OpEnum.OpEnum
import pillars.mapping.{DFG, DotReader, ILPMap, OmtMap, SearchMap}
import pillars.testers.{AppTestHelper, ApplicationTester}
import scala.util.Random
import java.lang.Math

## Initialize a simple CGRA

In [ ]:
class SimpleBlockWithReg(name: String, isRegion: Boolean = false) extends BlockTrait {
    initName(name)
    addInPorts(Array("input_0","input_1"))
    addOutPorts(Array("out_0"))
    
    if(isRegion){
        setConfigRegion()
    }
    
    // Initialize ALU supporting ADD/SUB
    val aluOpList = List(OpEnum.ADD, OpEnum.SUB)
    val supBypass = false 
    val aluParams = List(32) // 32 bit width
    val alu0 = new ElementAlu(name+"_ALU", aluOpList, supBypass, List(32))
    alu0.addInPorts(Array("input_A", "input_B"))
    alu0.addOutPorts(Array("out_0"))
    addElement(alu0)

    // A register file with 2 registers 
    val rf0 = new ElementRF("rf0", List(1, 1, 1, 32))
    //port sequnces outs: 0: out_0
    //port sequnces inputs: 0: input_0
    rf0.addOutPorts(Array("out_0"))
    rf0.addInPorts(Array("input_0"))
    addElement(rf0)
    
    // Initialize internal connections
    addConnect(term("input_0") -> alu0 / "input_A")
    addConnect(term("input_1") -> alu0 / "input_B")
    addConnect(alu0 / "out_0" -> rf0 / "input_0")
    addConnect(rf0 / "out_0" -> term("out_0"))
}


In [ ]:
val inputPort = 2
val outputPort = 1
val dataWidth = 32

//Initialize the top block.
val arch = new ArchitectureHierarchy()
arch.addInPorts((0 until inputPort).map(i => s"input_$i").toArray)
arch.addOutPorts((0 until outputPort).map(i => s"out_$i").toArray)

val block = new SimpleBlockWithReg("Block0")
arch.addBlock(block)

(0 until inputPort).foreach(i =>
    arch.addConnect(arch.term(s"input_$i") -> block / s"input_$i"))
(0 until outputPort).foreach(i =>
    arch.addConnect(block / s"out_$i" -> arch.term(s"out_$i")))
arch.init()

## Prepare a runtime infomation file for the simple Add DFG.
Some values can be customized:
1. The input data for an "input" op.
2. The expected data for a "output" op.
3. The const values for an "const" op.
4. The initial/step/end/freq values for a "incr" op to guide a loop.
5. The data stored in a SRAM before the CGRA starts.
6. The expected data in a SRAM after the CGRA stops.

In [ ]:
def prepareRuntimeInfoAdd(dfg: DFG) = {
      val dataSize = 10
      val A = (0 until dataSize).map(i => i).toList
      val B = (0 until dataSize).map(i => i).toList
      val expectedRet = (0 until dataSize).map(i => A(i) + B(i))

      //Please make sure there are 2 operators with INPUT opcode in the DFG.
      val inputOpNames = dfg.opNodes.filter(op => op.opcode == OpEnum.INPUT).map(op => op.name)
      val inputToPort = List(InputToPort(inputOpNames(0), A), InputToPort(inputOpNames(1), B))

      val outputOpNames = dfg.opNodes.filter(op => op.opcode == OpEnum.OUTPUT).map(op => op.name)
      val outputFromPort = List(OutputFromPort(outputOpNames(0), expectedRet.toList))

      val emptyList = List()

      val runtimeInfo = RuntimeInfo(inputToPort, outputFromPort, emptyList
        , emptyList, emptyList, emptyList)

      runtimeInfo
    }

viewDFG("./Add.dot.jpg")
val dfgFilename = "./Add.dot"
val II = 2
val dfg = DotReader.loadDot(dfgFilename, II)
prepareRuntimeInfoAdd(dfg)
JsonParser.writeJson(prepareRuntimeInfoAdd(dfg), "runtime.json")
val runtimeInfo = JsonParser.readJson("runtime.json", print = true)

## Use some helpful classes to initialize configs from mapping results

In [ ]:
//Simulation settings.
val simulationHelper = new SimulationHelper(arch)
val resultFilename = s"Add_ii$II" + "_r.txt"
simulationHelper.init(resultFilename, runtimeInfo, II, print = true)

val appTestHelper = new AppTestHelper(II)
val moduleInfoFilename = s"Add_ii$II" + "_i.txt"
appTestHelper.init(arch, simulationHelper, moduleInfoFilename, runtimeInfo, print = true)

## Generate configs from the mapping results between the Vadd-Reverse DFG and the 4X4 TileLSUBlock in library

In [ ]:
val rowNum = 4
val colNum = 4
val inputPort = 4
val outputPort = 4
val dataWidth = 32

//Initialize the top block.
val arch = new ArchitectureHierarchy()
arch.addInPorts((0 until inputPort).map(i => s"input_$i").toArray)
arch.addOutPorts((0 until outputPort).map(i => s"out_$i").toArray)

val tile = new TileLSUBlock("tile_0", colNum, rowNum, inputPort, outputPort,
    useMuxBypass = false, complex = true, isToroid = false, useCounter = true, dataWidth = dataWidth)
arch.addBlock(tile)

(0 until inputPort).foreach(i =>
    arch.addConnect(arch.term(s"input_$i") -> tile / s"input_$i"))
(0 until outputPort).foreach(i =>
    arch.addConnect(tile / s"out_$i" -> arch.term(s"out_$i")))
arch.init()

In [ ]:
 def prepareRuntimeInfo(dfg: DFG, numSRAM: Int) = {
      val dataSize = 50
      val VectorA = (0 until dataSize).map(_ => Math.abs(scala.util.Random.nextInt() % 1000)).toArray
      val VectorB = (0 until dataSize).map(_ => Math.abs(scala.util.Random.nextInt() % 1000)).toArray

      //Input random indexes into the mapped input port in CGRA,
      // and get A(index) + B(index) from the mapped output port.
      val inputIndexes = Random.shuffle((0 until dataSize).toList)
      val expectedRet = (0 until dataSize).map(i => VectorA(inputIndexes(i)) + VectorB(inputIndexes(i)))

      //The base address of A and B in SRAM of an LSU.
      //To simplify the problem, we assume both A and B are stored
      //in all SRAMs belonging to 4 LSUs in the targeted architecture.
      val a_base = 0
      val b_base = dataSize

      //The value of const operators.
      val const0 = a_base
      val const1 = b_base
      val const2 = dataSize - 1
      val const3 = a_base
      val const4 = a_base
      val constVals = Array(const0, const1, const2, const3, const4)

      val constOpNames = dfg.opNodes.filter(op => op.opcode == OpEnum.CONST).map(op => op.name)
      val constValue = (0 until constOpNames.size).map(i => ConstValue(constOpNames(i), constVals(i))).toList

      //Operator incr0 should generate (j <- 0 until dataSize).
      //So the parameter of the counter is (init = 0, step = 1, end = dataSize, freq = 1)
      val counterOpNames = dfg.opNodes.filter(op => op.opcode == OpEnum.INCR).map(op => op.name)
      val counterConfig = List(CounterConfig(counterOpNames(0), 0, 1, dataSize, 1))

      //In this simple tutorial, A and B are put into all LSUs.
      //But you can put them into partial LSUs according to the mapping results,
      // just like what in the ApplicationExamples.
      //Because the PEs in a row share an LSU, the number of LSUs is rowNum.
      val inputToSRAM = (0 until numSRAM).map(i => InputToSRAM(i, a_base, VectorA.toList)).toList :::
        (0 until numSRAM).map(i => InputToSRAM(i, b_base, VectorB.toList)).toList

      val outputFromSRAM = List(OutputFromSRAM(3, a_base, VectorA.reverse.toList))

      //Please make sure there are 2 operators with INPUT opcode in the DFG.
      val inputOpNames = dfg.opNodes.filter(op => op.opcode == OpEnum.INPUT).map(op => op.name)
      //      val inputToPortData = List(inputI, inputJ)
      //      val inputToPort = (0 until inputOpNames.size).map(i => InputToPort(inputOpNames(i),
      //        inputToPortData(i).toList)).toList
      val inputToPort = List(InputToPort(inputOpNames(0),
        inputIndexes))

      val outputOpNames = dfg.opNodes.filter(op => op.opcode == OpEnum.OUTPUT).map(op => op.name)
      //      val outputFromPort = List(OutputFromPort(outputOpNames(0), outResult.toList))
      val outputFromPort = List(OutputFromPort(outputOpNames(0), expectedRet.toList))

      val runtimeInfo = RuntimeInfo(inputToPort, outputFromPort, inputToSRAM
        , outputFromSRAM, constValue, counterConfig)

      runtimeInfo
    }

    
val II = 1
val dfgFilename = "Vadd_Reverse_.dot"
val dfg = DotReader.loadDot(dfgFilename, II)

viewDFG("./Vadd_Reverse_.dot.jpg")
JsonParser.writeJson(prepareRuntimeInfo(dfg, rowNum), "runtime.json")
val runtimeInfo = JsonParser.readJson("runtime.json", print = true)

//Simulation settings.
val simulationHelper = new SimulationHelper(arch)
val resultFilename = s"ii$II" + "_r.txt"
simulationHelper.init(resultFilename, runtimeInfo, II, print = true)

val appTestHelper = new AppTestHelper(II)
val moduleInfoFilename = s"ii$II" + "_i.txt"
appTestHelper.init(arch, simulationHelper, moduleInfoFilename, runtimeInfo, print = true)